In [ ]:
import pandas as pd
import numpy as np
from helpers import return_good_regions
import pickle
import matplotlib.pyplot as plt

In [ ]:
# Add other IQR
def add_hourly_iqr(df, hIQR):
    rolling = np.empty((0,), float)
    
    for i in range(len(df.index)):
        lst = df.loc[i-hIQR:i+hIQR-1, 'dem_rel_diff_wrt_hourly_delta_pre']
        iqr = np.nanpercentile(lst, 75) - np.nanpercentile(lst, 25)
        rolling = np.append(rolling, iqr)
    
    return df.assign(hourly_IQR=rolling)


# normalize IQRs
def df_normalize(df, cols):
    for col in cols:
        df[col] = df[col] / np.mean(df[col])
    return df



def print_stats(df, col):
    print(f"{col},{np.nanmax(df[col])},{np.nanmin(df[col])},{np.nanmean(df[col])},{np.std(df[col])}", end =",")


In [ ]:
regions = return_good_regions()
regions = ['CISO',]#'FPL']
version = '_v12_2day'
app = 'Oct15'

hIQR = 24*5
iqrs = ['dem_minus_rolling_IQR', 'delta_rolling_IQR', 'hourly_IQR']

for region in regions:
    #print('Loading from pickle /Users/truggles/tmp_data{}/pickle_{}{}_r2.pkl'.format(app, region, version))
    pickle_in = open('/Users/truggles/tmp_data{}/pickle_{}{}_r2.pkl'.format(app, region, version),'rb')
    df = pickle.load(pickle_in)
    df['date_time'] = pd.to_datetime(df['date_time'])
    
   
    df = add_hourly_iqr(df, hIQR)

#cols = df.columns
#for col in cols:
#    if 'IQR' in col or 'iqr' in col:
#        print(col)
#print(cols)


    df = df_normalize(df, iqrs)

    print(region, end =",")
    for col in iqrs:
        print_stats(df, col)
    print("")

In [ ]:
cols = df.columns
for col in cols:
    if 'IQR' in col or 'iqr' in col:
        print(col)
print(cols)

In [ ]:
fig, ax = plt.subplots()
ax.plot(df['dem_minus_rolling_IQR'])
ax.plot(df['delta_rolling_IQR'])
ax.plot(df['hourly_IQR'])